# LangGraph Fundamentals

Welcome to this comprehensive guide on **LangGraph**, a powerful framework for building stateful, multi-agent applications with LLMs. 

LangGraph allows you to orchestrate complex AI workflows by organizing your application as a **graph** - where nodes represent actions (like calling an LLM or executing a tool) and edges define how data flows between them.

In this notebook, you'll learn the **7 core building blocks** that make up every LangGraph application:

1. **State** - The data container that flows through your graph
2. **Nodes** - Functions that perform work and update the state
3. **Edges** - Connections that define the flow between nodes
4. **Reducers** - Strategies for merging state updates from parallel nodes
5. **Tools** - Functions that LLMs can call to interact with external systems
6. **Memory (Checkpointers)** - Persistence layer to save and resume workflows
7. **Routing Functions** - Logic to dynamically choose paths through your graph
8. **Superstep** - one complete round of execution 

By the end of this notebook, you'll understand how these components work together and be able to build your own agent workflows - from simple chatbots to complex multi-agent systems.

Let's get started! 🚀

---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).

In [ ]:
# Standard library imports
import os
import operator
import sqlite3
from typing import TypedDict, Annotated, Literal

# Third-party imports
import gradio as gr
from dotenv import load_dotenv
from IPython.display import Image, display

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode, tools_condition


## 🗺️ State, Nodes, and Edges

Think of LangGraph like a road trip:

- State = Your car (carries data like name, count, messages)
- Nodes = Cities where you stop and do something (greet someone, calculate, call a tool)
- Edges = Roads connecting the cities (define which city comes next)

In [ ]:
# STATE = Your car carrying data
class State(TypedDict):
    message: str

# NODES = Cities where work happens
def city_a(state: State):
    print("🏙️ City A: Processing...")
    return {"message": state["message"] + " → visited A"}

def city_b(state: State):
    print("🏙️ City B: Processing...")
    return {"message": state["message"] + " → visited B"}

# BUILD THE MAP
graph = StateGraph(State)
graph.add_node("A", city_a)
graph.add_node("B", city_b)

# EDGES = Roads between cities
graph.add_edge(START, "A")  # Start → City A
graph.add_edge("A", "B")     # City A → City B
graph.add_edge("B", END)     # City B → End

app = graph.compile()

# TAKE THE TRIP
result = app.invoke({"message": "Starting"})
print(f"\n📍 Final: {result['message']}")

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))

## 🔀 Reducer

You need a reducer (which is a function) when multiple nodes want to update the SAME field in the state, and you want to combine/merge their updates instead of having the last node overwrite everyone else's work.

**Common cases:**

1. **Multiple agents collecting data** - Each agent finds something and you want to keep ALL findings, not just the last one
   - Example: 3 researchers each add their findings to a list

2. **Accumulating messages** - Multiple nodes adding messages to a conversation history
   - Example: User message + Agent 1 response + Agent 2 response = full conversation

3. **Aggregating scores/metrics** - Multiple nodes calculating partial results
   - Example: Sum up scores from different evaluators

4. **Parallel processing** - When nodes run in parallel and all contribute to the same field

**Without reducer:** Last node wins, previous updates are lost ❌  
**With reducer:** All updates are combined/merged ✅

**`add_messages`** is the most common for chat applications (handles message-specific logic like deduplication), while **`operator.add`** is the most common for general list appending in any other use case.

In [ ]:
# ========== STATE WITH operator.add REDUCER ==========
class State(TypedDict):
    # WITH reducer: values accumulate ✅
    scores: Annotated[list[int], operator.add]

# ========== TWO NODES THAT RUN IN PARALLEL ==========
def node_a(state: State):
    print("Node A running...")
    return {"scores": [10, 20]}

def node_b(state: State):
    print("Node B running...")
    return {"scores": [30, 40]}

# ========== BUILD GRAPH ==========
graph = StateGraph(State)
graph.add_node("a", node_a)
graph.add_node("b", node_b)

# Both run in parallel from START
graph.add_edge(START, "a")
graph.add_edge(START, "b")
graph.add_edge("a", END)
graph.add_edge("b", END)

app = graph.compile()

# ========== RUN ==========
result = app.invoke({"scores": []})

print("\nScores (with operator.add reducer):", result["scores"])

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))

In [ ]:
# ========== STATE FOR AGENT ==========
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# ========== AGENT NODES ==========
def analyzer_agent(state: AgentState):
    """Analyzes user input"""
    last_message = state["messages"][-1].content

    return {
        "messages": [AIMessage(content=f"Analysis: '{last_message}' looks like a question")]
    }

def responder_agent(state: AgentState):
    """Responds based on analysis"""
    return {
        "messages": [AIMessage(content="Here's my answer based on the analysis...")]
    }

# ========== BUILD GRAPH ==========
graph = StateGraph(AgentState)
graph.add_node("analyze", analyzer_agent)
graph.add_node("respond", responder_agent)

graph.add_edge(START, "analyze")
graph.add_edge("analyze", "respond")
graph.add_edge("respond", END)

app = graph.compile()

# ========== RUN ==========
result = app.invoke({
    "messages": [HumanMessage(content="What is LangGraph?")]
})

print("Full conversation:")
for msg in result["messages"]:
    print(f"  [{msg.type}]: {msg.content}")

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))

## 🛠️ Tools
Tools are functions that agents can call to perform specific tasks

In [ ]:
# ========== SIMPLE TOOL ==========
@tool
def send_notification(message: str) -> str:
    """Send a notification with a message"""
    print(f"📩 Notification sent: {message}")
    return f"Sent: {message}"

# ========== STATE ==========
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ========== LLM WITH TOOLS ==========
tools = [send_notification]
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

# ========== NODES ==========
def chatbot(state: State):
    """LLM decides whether to use tools"""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools)

# ========== BUILD GRAPH ==========
graph = StateGraph(State)
graph.add_node("chatbot", chatbot)
graph.add_node("tools", tool_node)

graph.add_edge(START, "chatbot")
graph.add_conditional_edges("chatbot", tools_condition)
graph.add_edge("tools", "chatbot")

app = graph.compile()

# ========== RUN ==========
result = app.invoke({
    "messages": [("user", "Send me a notification saying 'Hello World'")]
})

print("\nFinal messages:")
for msg in result["messages"]:
    print(f"  {msg.type}: {msg.content}")

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))


## 💾 Memory

- Memory (checkpointer) allows the chatbot to remember past conversations:
    - SqliteSaver stores conversation history in a database file (memory.db)
    - thread_id identifies different conversation sessions (like a chat room ID)
    - Same thread_id = same conversation history
    - Different thread_id = separate, independent conversations
    - When you restart the code, it loads the saved history from the database
    - add_messages reducer ensures messages are appended (not overwritten)

In [ ]:
load_dotenv()

# State
class State(TypedDict):
    messages: Annotated[list, add_messages]

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Node
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]} # Pass the entire conversation history

# Graph with SQLite
graph = StateGraph(State)
graph.add_node("chatbot", chatbot)
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

conn = sqlite3.connect("memory.db", check_same_thread=False)
memory = SqliteSaver(conn)
app = graph.compile(checkpointer=memory)

# Gradio
def chat(message, _):
    config = {"configurable": {"thread_id": "1"}}
    response = app.invoke({"messages": [("user", message)]}, config) # Runs the entire graph workflow
    return response["messages"][-1].content

demo = gr.ChatInterface(chat, title="LangGraph Chatbot with Memory")
demo.launch()

## 🚦 Routing Functions
A routing function is like a traffic light that decides which path to take based on conditions.

In [ ]:
# STATE
class State(TypedDict):
    question: str
    info_collected: list[str]
    answer: str

# NODES
def search_page(state: State):
    """Scrape a webpage"""
    print("🔍 Scraping webpage...")
    new_info = f"Info from page {len(state['info_collected']) + 1}"
    return {"info_collected": state["info_collected"] + [new_info]}

def give_answer(state: State):
    """Provide final answer"""
    print("✅ Enough info! Giving answer...")
    return {"answer": f"Answer based on {len(state['info_collected'])} pages"}

# ROUTING FUNCTION - Do we have enough info?
def check_if_enough(state: State) -> Literal["search_more", "answer"]:
    """Decide: need more info or ready to answer?"""
    if len(state["info_collected"]) < 3:
        print(f"⚠️ Only {len(state['info_collected'])} pages. Need more!")
        return "search_more"
    else:
        print(f"✅ Got {len(state['info_collected'])} pages. Ready!")
        return "answer"

# BUILD GRAPH
graph = StateGraph(State)
graph.add_node("search", search_page)
graph.add_node("answer", give_answer)

graph.add_edge(START, "search")

# CONDITIONAL EDGE - Router decides: more scraping or answer?
graph.add_conditional_edges(
    "search",
    check_if_enough,
    {"search_more": "search", "answer": "answer"}  # Loop back or finish
)

graph.add_edge("answer", END)

app = graph.compile()

# RUN
result = app.invoke({
    "question": "What is LangGraph?",
    "info_collected": [],
    "answer": ""
})

print(f"\n📄 Pages scraped: {len(result['info_collected'])}")
print(f"💬 Answer: {result['answer']}")

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))

## ⚡ Superstep

Here's a SIMPLE analogy:

**Making a Sandwich (Sequential)**

**Steps:**
1. Get bread
2. Add peanut butter
3. Add jelly
4. Close sandwich

**Each step happens ONE AT A TIME** (you can't add jelly before bread!)

**In LangGraph terms:**
- Each step = 1 node
- Since they're sequential, each is also called a "superstep"
- **3 steps = 3 supersteps**

---

**Making 3 Sandwiches with 3 People (Parallel)**

**Superstep 1:** (Everyone works at SAME TIME)
- Person A gets bread
- Person B gets bread
- Person C gets bread
**(3 steps happen in 1 superstep!)**

**Superstep 2:** (Everyone waits for everyone to finish, then all work together)
- Person A adds peanut butter
- Person B adds peanut butter  
- Person C adds peanut butter
**(3 steps happen in 1 superstep!)**

**Superstep 3:**
- Person A adds jelly
- Person B adds jelly
- Person C adds jelly

---

**The Rule:**

**Superstep = A "round" where everyone works, then everyone waits**

- **Sequential** → 1 person working = 1 step = 1 superstep
- **Parallel** → 3 people working = 3 steps = 1 superstep

---

A superstep is **one complete** round of execution in LangGraph where:

- All nodes that are ready to run execute (potentially in parallel)
- Their updates get applied to the state
- LangGraph checks what to do next

In [ ]:
class State(TypedDict):
    count: int

def step_1(state: State):
    print(f"Step 1 (superstep 1) - count: {state['count']}")
    return {"count": state["count"] + 1}

def step_2(state: State):
    print(f"Step 2 (superstep 2) - count: {state['count']}")
    return {"count": state["count"] + 1}

def step_3(state: State):
    print(f"Step 3 (superstep 3) - count: {state['count']}")
    return {"count": state["count"] + 1}

graph = StateGraph(State)
graph.add_node("s1", step_1)
graph.add_node("s2", step_2)
graph.add_node("s3", step_3)

graph.add_edge(START, "s1")
graph.add_edge("s1", "s2")
graph.add_edge("s2", "s3")
graph.add_edge("s3", END)

app = graph.compile()

result = app.invoke({"count": 0})

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))